In [ ]:
#TO DO:
#fix formatting - do this on new copy of df's 

#QUESTIONS
#is total students sum of unique values of sizes of schools or count of students in dataset? 

In [ ]:
# Dependencies and Setup
import pandas as pd

# File to Load 
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_data_complete.head()

In [ ]:
#District Summary
#Total Number of Schools
totalSchools = len(school_data_complete['school_name'].unique())

#Total Number of Students
totalStudents = len(school_data_complete['Student ID'])

#Total Budget
totalBudget = sum(school_data_complete['budget'].unique())

#Average Math Score
avgMath = school_data_complete['math_score'].mean()

#Average Reading Score
avgReading = school_data_complete['reading_score'].mean()

#Percentage Passing Math (70 or greater)
totalPassingMath = school_data_complete.loc[school_data_complete['math_score']>=70, 'Student ID'].count()

percPassingMath = totalPassingMath/totalStudents*100

#Percentage Passing Reading (70 or greater)
totalPassingRead = school_data_complete.loc[school_data_complete['reading_score']>=70, 'Student ID'].count()

percPassingRead = totalPassingRead/totalStudents*100

#Percentage Overall Passing
passingOnly = school_data_complete.loc[(school_data_complete['math_score']>=70)&(school_data_complete['reading_score']>=70), 'Student ID'].count()
percPassOnly = passingOnly/totalStudents*100

#Dataframe of results
district_summary_df = pd.DataFrame({
    'Total Schools': [totalSchools],
    'Total Students': [totalStudents],
    'Total Budget': [totalBudget],
    'Average Math Score': [avgMath],
    'Average Reading Score': [avgReading],
    '% Passing Math': [percPassingMath],
    '% Passing Reading': [percPassingRead],
    '% Overall Pass': [percPassOnly]
})

district_summary_df

In [ ]:
#School Summary

#Group by school 
school_data_byschool = school_data_complete.groupby(['school_name'])

#Determine total students that were tested in each school
#Did not use size of school because we cannot assume that all students were tested
totalStudentsTested = school_data_byschool['Student ID'].count()

#Average Math Score
avgMathSchool = school_data_byschool['math_score'].mean().rename('Average Math Score')

#Average Reading Score
avgReadingSchool = school_data_byschool['reading_score'].mean().rename('Average Reading Score')

#Percent Passing Math

allPassingMath = school_data_complete.loc[school_data_complete['math_score']>=70]
percPassingMath = allPassingMath.groupby(['school_name']).count()['math_score']/totalStudentsTested*100

#Percent Passing Reading
allPassingReading = school_data_complete.loc[school_data_complete['reading_score']>=70]
percPassingReading = allPassingReading.groupby(['school_name']).count()['reading_score']/totalStudentsTested*100


#Overall Passing
#for all student math scores above 70 and reading scores above 70, sum them and divide by total students
overallPassing = school_data_complete.loc[(school_data_complete['math_score']>=70) 
                                          & (school_data_complete['reading_score']>=70)]
percOverallPassing = overallPassing.groupby(['school_name']).count()['Student ID']/totalStudentsSchool*100

#Pull desired columns from original dataset and merge with totals and averages
schoolSummary1 = school_data[['school_name', 'type', 'size', 'budget']].set_index('school_name')
schoolSummary2 = pd.concat([
    avgMathSchool, 
    avgReadingSchool, 
    percPassingMath, 
    percPassingReading,
    percOverallPassing
], axis=1)
schoolSummary3 = pd.merge(schoolSummary1, schoolSummary2, left_index=True, right_index=True)

schoolSummary3['Per Student Budget'] = schoolSummary3['budget']/schoolSummary3['size']

#Format DataFrame
schoolSummary4 = schoolSummary3.rename(columns={
    'type': 'School Type',
    'size': 'Total Students',
    'budget': 'School Budget',
    0: '% Passing Math',
    1: '% Passing Reading',
    2: '% Overall Passing'
})

schoolSummaryFinal = schoolSummary4[[
    'School Type', 
    'Total Students', 
    'School Budget', 
    'Per Student Budget',
    'Average Math Score',
    'Average Reading Score',
    '% Passing Math',
    '% Passing Reading',
    '% Overall Passing'
]]

schoolSummaryFinal.sort_index()

In [ ]:
#Top 5 Performing Schools (By % Overall Passing)
schoolSummaryFinal.sort_values(by=['% Overall Passing'], ascending=False).head(5)

In [ ]:
#Bottom 5 Performing Schools (By % Overall Passing)
schoolSummaryFinal.sort_values(by=['% Overall Passing']).head(5)

In [ ]:
#Math Scores by Grade

#Create a series for each grade
ninthGrade = school_data_complete.loc[school_data_complete['grade']=='9th']
tenthGrade = school_data_complete.loc[school_data_complete['grade']=='10th']
eleventhGrade = school_data_complete.loc[school_data_complete['grade']=='11th']
twelfthGrade = school_data_complete.loc[school_data_complete['grade']=='12th']

#Group each series by school
ninthSchool = ninthGrade.groupby(['school_name']).mean()['math_score']
ninthSeries = pd.Series(ninthSchool, name='9th')
tenthSchool = tenthGrade.groupby(['school_name']).mean()['math_score']
tenthSeries = pd.Series(tenthSchool, name='10th')
eleventhSchool = eleventhGrade.groupby(['school_name']).mean()['math_score']
eleventhSeries = pd.Series(eleventhSchool, name='11th')
twelfthSchool = twelfthGrade.groupby(['school_name']).mean()['math_score']
twelfthSeries = pd.Series(twelfthSchool, name='12th')


#Combine the series into a dataframe
mathByGrade = pd.concat([ninthSeries, tenthSeries, eleventhSeries, twelfthSeries], axis=1)
mathByGrade

In [ ]:
#Reading Scores by Grade

#Create a series for each grade
ninthGrade = school_data_complete.loc[school_data_complete['grade']=='9th']
tenthGrade = school_data_complete.loc[school_data_complete['grade']=='10th']
eleventhGrade = school_data_complete.loc[school_data_complete['grade']=='11th']
twelfthGrade = school_data_complete.loc[school_data_complete['grade']=='12th']

#Group each series by school
ninthSchoolR = ninthGrade.groupby(['school_name']).mean()['reading_score']
ninthSeriesR = pd.Series(ninthSchool, name='9th')
tenthSchoolR = tenthGrade.groupby(['school_name']).mean()['reading_score']
tenthSeriesR = pd.Series(tenthSchool, name='10th')
eleventhSchoolR = eleventhGrade.groupby(['school_name']).mean()['reading_score']
eleventhSeriesR = pd.Series(eleventhSchool, name='11th')
twelfthSchoolR = twelfthGrade.groupby(['school_name']).mean()['reading_score']
twelfthSeriesR = pd.Series(twelfthSchool, name='12th')


#Combine the series into a dataframe
readByGrade = pd.concat([ninthSeriesR, tenthSeriesR, eleventhSeriesR, twelfthSeriesR], axis=1)
readByGrade

In [ ]:
#Scores by School Spending

#Create bins
schoolSummarySpending = schoolSummaryFinal.copy()
budgetLabels = ['<$584', '$585-629', '$630-644', '$645-675']
budgetBins = [0, 585, 630, 645, 675]
schoolSummarySpending['Spending Ranges (Per Student)'] = pd.cut(schoolSummarySpending['Per Student Budget'], bins=budgetBins, labels=budgetLabels)

#Group by bins and calculate average
avgBudgetBin = schoolSummarySpending.groupby(schoolSummarySpending['Spending Ranges (Per Student)']).mean()

#Create dataframe with desired columns
budgetBin_df = avgBudgetBin[[
    'Average Math Score', 
    'Average Reading Score', 
    '% Passing Math', 
    '% Passing Reading', 
    '% Overall Passing'
]]

budgetBin_df

In [ ]:
#Scores by School Size

#Create bins
schoolSummarySize = schoolSummaryFinal.copy()
sizeLabels = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']
sizeBins = [0, 1000, 2000, 5000]
schoolSummarySize['School Size'] = pd.cut(schoolSummarySize['Total Students'], bins=sizeBins, labels=sizeLabels)

#Group by bins and calculate average
avgSizeBin = schoolSummarySize.groupby(schoolSummarySize['School Size']).mean()

#Create dataframe with desired columns
sizeBin_df = avgSizeBin[[
    'Average Math Score', 
    'Average Reading Score', 
    '% Passing Math', 
    '% Passing Reading', 
    '% Overall Passing'
]]

sizeBin_df

In [ ]:
#Scores by School Type
#Group by school type and calculate averages
schoolSummaryType = schoolSummaryFinal.copy()
typeGrouped = schoolSummarySize.groupby(schoolSummarySize['School Type']).mean()

#Create dataframe with desired columns
typeGrouped_df = typeGrouped[[
    'Average Math Score', 
    'Average Reading Score', 
    '% Passing Math', 
    '% Passing Reading', 
    '% Overall Passing'
]]

typeGrouped_df